In [ ]:
# Create a mask overlay with purple color (Red + Blue)
mask_overlay = np.zeros_like(rgb)
mask_overlay[:, :, 0] = 128  # Red channel for purple
mask_overlay[:, :, 1] = 0  # Green channel for purple
mask_overlay[:, :, 2] = 128  # Blue channel for purple

# Apply the filtered mask to the mask overlay
mask_overlay[closing != 1] = [0, 0, 0]  # Transparent where mask is not applied


In [ ]:
# Create the inverse mask
inverse_closing = np.logical_not(closing).astype(np.uint8)

# Create an inverse mask overlay with purple color
inverse_mask_overlay = np.zeros_like(rgb)
inverse_mask_overlay[:, :, 0] = 128  # Red channel for purple
inverse_mask_overlay[:, :, 1] = 0  # Green channel for purple
inverse_mask_overlay[:, :, 2] = 128  # Blue channel for purple

# Apply the inverse mask to the inverse mask overlay
inverse_mask_overlay[closing == 1] = [
    0, 0, 0
]  # Transparent where original mask is applied


In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(18, 6))

plt.sca(ax[0])
plt.imshow(rgb)
plt.axis('off')
plt.title('RGB')

plt.sca(ax[1])
plt.imshow(rgb)
plt.imshow(mask_overlay, alpha=0.5)
plt.axis('off')
plt.title('vegetation segmentation')

plt.sca(ax[2])
plt.imshow(rgb)
plt.imshow(inverse_mask_overlay, alpha=0.5)
plt.axis('off')
plt.title('non-veg segmentation')

plt.show()